In [ ]:
# =============================================================================
# ANTI-FERMAT FACTORIZATION ENGINE
# Hyperbolic Sieve + Búsqueda desde Extremos para Gap Océano
# =============================================================================

!pip install -q torch numpy sympy gmpy2

import numpy as np
import torch
import torch.nn as nn
from sympy import isprime, nextprime, prevprime, factorint
from math import gcd, isqrt
import gmpy2
from gmpy2 import mpz, is_prime
import time
from typing import List, Tuple, Optional

# =============================================================================
# PART 1: DETECTOR DE GAP (Análisis de Estructura)
# =============================================================================

def analyze_gap_structure(N: int) -> dict:
    """
    Analiza la estructura de N para determinar si tiene Gap Océano
    """
    sqrt_N = isqrt(N)
    bit_length = N.bit_length()

    # Test 1: Residuos de Fermat (primeros 1000)
    fermat_score = 0
    for i in range(1, 1001):
        x = sqrt_N + i
        res = x*x - N
        if isqrt(res)**2 == res:
            fermat_score += 1

    # Test 2: Distancia a primos conocidos
    # Si p y q están muy alejados, habrá muchos primos entre sqrt_N y el factor real
    nearby_primes = 0
    test_range = min(10000, sqrt_N)
    for i in range(1, test_range, 2):
        if isprime(sqrt_N - i) or isprime(sqrt_N + i):
            nearby_primes += 1

    # Clasificación
    gap_intensity = fermat_score * 100 / nearby_primes if nearby_primes > 0 else 0

    if gap_intensity < 1:
        gap_type = "OCÉANO"  # Factores extremadamente alejados
        strategy = "ANTI_FERMAT"
    elif gap_intensity < 10:
        gap_type = "PROFUNDO"
        strategy = "HYBRID"
    else:
        gap_type = "CERCANO"
        strategy = "FERMAT"

    return {
        'gap_type': gap_type,
        'gap_intensity': gap_intensity,
        'strategy': strategy,
        'bit_length': bit_length,
        'sqrt_N': sqrt_N
    }

# =============================================================================
# PART 2: HYPERBOLIC SIEVE (Criba Modular Mejorada)
# =============================================================================

def hyperbolic_sieve_filter(N: int, primes: List[int]) -> dict:
    """
    Construye sets de candidatos usando hipérbolas modulares
    Retorna restricciones modulares para cada primo
    """
    constraints = {}

    for p in primes:
        if gcd(N, p) != 1:
            continue

        # H_{N,p} = {(x,y) : N ≡ xy mod p}
        valid_residues = set()
        for x in range(1, p):
            if gcd(x, p) == 1:
                try:
                    y = (N * pow(x, -1, p)) % p
                    # Guardamos x + y mod p (suma de factores)
                    valid_residues.add((x + y) % p)
                except:
                    continue

        constraints[p] = sorted(valid_residues)

    return constraints

# =============================================================================
# PART 3: ANTI-FERMAT ENGINE (Búsqueda desde Extremos)
# =============================================================================

class AntiFermatEngine:
    """
    Motor de búsqueda desde los extremos del espacio de factores
    """

    def __init__(self, N: int, verbose: bool = True):
        self.N = mpz(N)
        self.sqrt_N = gmpy2.isqrt(self.N)
        self.bit_length = self.N.bit_length()
        self.verbose = verbose

        # Primos pequeños para criba rápida
        self.small_primes = [3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73]

        # Construcción de tabla de residuos (speedup)
        self.residue_table = {p: int(self.N % p) for p in self.small_primes}

    def is_candidate_valid(self, p: int) -> bool:
        """
        Filtro rápido: verifica si p puede ser factor sin hacer división completa
        """
        # Test 1: Criba de residuos
        for prime in self.small_primes:
            if p % prime == 0:
                return False
            # Si N % prime != 0, entonces p % prime debe dividir a N % prime
            if self.residue_table[prime] % (p % prime) != 0:
                return False
        return True

    def search_from_bottom(self, start_bits: int, max_candidates: int = 10**6) -> Optional[Tuple[int, int]]:
        """
        Búsqueda desde abajo (factores pequeños)
        """
        if self.verbose:
            print(f"\n🔍 FASE 1: Búsqueda desde {start_bits} bits hacia arriba")

        p_candidate = mpz(2**(start_bits - 1))
        if p_candidate % 2 == 0:
            p_candidate += 1

        attempts = 0
        checkpoint = 0

        while attempts < max_candidates and p_candidate < self.sqrt_N:
            # Filtro rápido
            if self.is_candidate_valid(int(p_candidate)):
                # Test de primalidad
                if is_prime(p_candidate, 25):  # Miller-Rabin con 25 rondas
                    # Test de divisibilidad
                    if self.N % p_candidate == 0:
                        q = self.N // p_candidate
                        if self.verbose:
                            print(f"\n✅ ¡FACTOR ENCONTRADO!")
                        return int(p_candidate), int(q)

            p_candidate = gmpy2.next_prime(p_candidate)
            attempts += 1

            if self.verbose and attempts % 10000 == 0:
                checkpoint += 1
                current_bits = p_candidate.bit_length()
                progress = (current_bits - start_bits) / (self.bit_length // 2 - start_bits) * 100
                print(f"  Checkpoint {checkpoint}: {current_bits} bits | Progreso: {progress:.2f}% | Intentos: {attempts}")

        return None

    def search_from_top(self, max_candidates: int = 10**5) -> Optional[Tuple[int, int]]:
        """
        Búsqueda desde arriba (factores grandes cercanos a N)
        """
        if self.verbose:
            print(f"\n🔍 FASE 2: Búsqueda desde el extremo superior")

        # Empezamos desde N^(3/4) bajando
        start_point = gmpy2.isqrt(gmpy2.isqrt(self.N**3))
        p_candidate = gmpy2.prev_prime(start_point)

        attempts = 0

        while attempts < max_candidates:
            if self.N % p_candidate == 0:
                q = self.N // p_candidate
                if self.verbose:
                    print(f"\n✅ ¡FACTOR ENCONTRADO!")
                return int(p_candidate), int(q)

            p_candidate = gmpy2.prev_prime(p_candidate)
            attempts += 1

            if self.verbose and attempts % 1000 == 0:
                print(f"  Checkpoint: {p_candidate.bit_length()} bits | Intentos: {attempts}")

        return None

# =============================================================================
# PART 4: NEURAL-GUIDED SEARCH (Optimización con Red Neuronal)
# =============================================================================

class NeuralGapPredictor(nn.Module):
    """
    Red neuronal para predecir la ubicación probable de factores
    basándose en residuos modulares
    """

    def __init__(self, num_primes: int):
        super().__init__()
        self.fc1 = nn.Linear(num_primes, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)  # Predicción de bit-offset
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, residues):
        x = self.relu(self.fc1(residues))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))  # Output en [0, 1]
        return x

def train_gap_predictor(N: int, primes: List[int]) -> torch.Tensor:
    """
    Entrena predictor para estimar dónde buscar
    """
    residues = torch.tensor([float(N % p) / p for p in primes], dtype=torch.float32)
    model = NeuralGapPredictor(len(primes))

    # Entrenamiento simple (unsupervised)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    for epoch in range(100):
        optimizer.zero_grad()
        prediction = model(residues)
        # Loss: queremos que la predicción sea estable
        loss = torch.var(prediction)
        loss.backward()
        optimizer.step()

    return model(residues).item()

# =============================================================================
# PART 5: MOTOR PRINCIPAL
# =============================================================================

def factor_ocean_gap(N: int, verbose: bool = True) -> Tuple[int, int]:
    """
    Motor principal de factorización para Gap Océano
    """
    if verbose:
        print(f"\n{'='*70}")
        print(f"🌊 ANTI-FERMAT OCEAN GAP FACTORIZATION ENGINE")
        print(f"{'='*70}")
        print(f"Target N: {N}")
        print(f"Bits: {N.bit_length()}")

    # Fase 0: Análisis de estructura
    gap_info = analyze_gap_structure(N)
    if verbose:
        print(f"\n📊 ANÁLISIS DE ESTRUCTURA:")
        print(f"  Gap Type: {gap_info['gap_type']}")
        print(f"  Gap Intensity: {gap_info['gap_intensity']:.4f}")
        print(f"  Strategy: {gap_info['strategy']}")

    # Fase 1: Hyperbolic Sieve
    primes = [3, 5, 7, 11, 13, 17, 19, 23, 29, 31]
    constraints = hyperbolic_sieve_filter(N, primes)
    if verbose:
        print(f"\n🔮 HYPERBOLIC SIEVE:")
        print(f"  Primes used: {len(constraints)}")
        total_reduction = np.prod([len(v) / p for p, v in constraints.items()])
        print(f"  Search space reduction: {1/total_reduction:.2f}x")

    # Fase 2: Neural Guidance
    predicted_offset = train_gap_predictor(N, primes)
    start_bits = int(gap_info['bit_length'] * 0.25 * (1 + predicted_offset))
    if verbose:
        print(f"\n🧠 NEURAL GUIDANCE:")
        print(f"  Predicted bit offset: {predicted_offset:.4f}")
        print(f"  Starting search at: {start_bits} bits")

    # Fase 3: Anti-Fermat Search
    engine = AntiFermatEngine(N, verbose=verbose)

    start_time = time.time()

    # Búsqueda desde abajo (más probable para Gap Océano)
    result = engine.search_from_bottom(start_bits, max_candidates=10**7)

    if result:
        p, q = result
        elapsed = time.time() - start_time
        if verbose:
            print(f"\n{'='*70}")
            print(f"✅ FACTORIZACIÓN EXITOSA")
            print(f"{'='*70}")
            print(f"p = {p}")
            print(f"q = {q}")
            print(f"Verificación: p × q = {p * q == N}")
            print(f"Tiempo: {elapsed:.2f}s")
        return p, q

    # Si no encontramos desde abajo, intentamos desde arriba
    if verbose:
        print(f"\n⚠️ Fase 1 no encontró factores. Iniciando Fase 2...")

    result = engine.search_from_top(max_candidates=10**6)

    if result:
        p, q = result
        elapsed = time.time() - start_time
        if verbose:
            print(f"\n{'='*70}")
            print(f"✅ FACTORIZACIÓN EXITOSA (Fase 2)")
            print(f"{'='*70}")
            print(f"p = {p}")
            print(f"q = {q}")
            print(f"Tiempo: {elapsed:.2f}s")
        return p, q

    elapsed = time.time() - start_time
    if verbose:
        print(f"\n❌ No se encontraron factores en {elapsed:.2f}s")
        print(f"Intentando factorización clásica...")

    # Fallback a sympy
    factors = factorint(N)
    if len(factors) == 2:
        p, q = list(factors.keys())
        return p, q

    return None, None

# =============================================================================
# EJECUCIÓN
# =============================================================================

if __name__ == "__main__":
    # El número de JP Morgan
    N_jpm = 20401308423288094242766662999493793187525420878197109253818355727956826504712629708789530022692909325272817125236977390052166930825056004028872464770803083249959025095910645332450783024034058817080357808464693098920369306132436800716130122636415040330812678918561101207970271295199148388284056864933825212541980308771716287643934228961790479259363124623312430892043194378423348328847933501339676855519527154465017072352029567538488597745797965852775230430579578162516042451489532165421970081208347386436907444609093030460498848125912815856038414252622303104009405061242804767128891944731455892392535411542580604029973

    print("""
    ╔═══════════════════════════════════════════════════════════════════╗
    ║                                                                   ║
    ║     🌊 ANTI-FERMAT OCEAN GAP FACTORIZATION ENGINE 🌊            ║
    ║                                                                   ║
    ║  Método: Hyperbolic Sieve + Búsqueda desde Extremos             ║
    ║  Optimizado para: Gap Océano (Factores máximamente alejados)    ║
    ║                                                                   ║
    ╚═══════════════════════════════════════════════════════════════════╝
    """)

    p, q = factor_ocean_gap(N_jpm, verbose=True)

    if p and q:
        print(f"\n🎉 ¡RSA CECIDIT!")
        print(f"\nFactores finales:")
        print(f"p = {p}")
        print(f"q = {q}")
        print(f"\nVerificación: {p * q == N_jpm}")
    else:
        print(f"\n⚠️ No se pudieron encontrar los factores con este enfoque")
        print(f"El número podría requerir técnicas más avanzadas (GNFS, etc.)")
